In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("./../../")
import rclpy as rp
import jupyros.ros2 as jr2
import jupyros.ros2.turtle_sim as turtle
import os

The rospy package is not found in your $PYTHONPATH. Subscribe and publish are not going to work.
Do you need to activate your ROS environment?


In [3]:
from time import time

In [4]:
rp.init()
rp.create_node("superturtle")

# Turtlesim

In [5]:
turtlesim = turtle.TurtleSim(background_color="#0000FF")
display(turtlesim.canvas)

turtle1 has spawned.


MultiCanvas(height=1600, layout=Layout(width='100%'), width=1600)

In [9]:
# Canvas default size is 1600x1600 starting from top-left corner
turtlesim.spawn(name="simon", pose={"x": 600, "y": 350, "theta": 20})

turtlesim.spawn(pose={"x": 1050, "y": 1250, "theta": 90})

# Change the turtle path color
turtlesim.turtles["turtle1"].path_color = "#F2F2F2"

NameError: name 'turtlesim' is not defined

In [ ]:
# !rosrun turtlesim turtlesim_node
# !rosrun turtlesim turtle_teleop_key

### Subscriber

In [ ]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

In [ ]:
topic_name = '/turtle1/pose'

# Based on the dimensions of window controlled by teleop keyboard
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * turtlesim.canvas.width
    y_convert = (convert_factor - y) / convert_factor * turtlesim.canvas.height
    
    return x_convert, y_convert

def callback_move_turtles(msg):
    name = "turtle1"
    x, y = convert_xy_units(msg.x, msg.y)
    poses[name] = {"x": x,
                   "y": y,
                   "theta": msg.theta}
    
    turtlesim.move_turtles(new_poses=poses)

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtles(msg))

### Spiral Publisher

In [ ]:
run = True

In [ ]:
%%thread_cell

rate = rospy.Rate(25)
pub = rospy.Publisher(topic_name, Pose, queue_size=10, latch=True)
i = 0

while run:
    msg = Pose()
    msg.x = (i/1800*5)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.publish(msg)
    rate.sleep()
    i += 1
print("Done")

In [ ]:
run = False

## Spawn Service and Client

In [ ]:
def spawn_turtle(msg):
    print(f"Spawning turtle at x={msg.x} and y={msg.y}")
    turtlesim.spawn(name=msg.name, 
                    pose={"x": msg.x, 
                          "y": msg.y, 
                          "theta": msg.theta})
    return "Spawned"

In [ ]:
%%thread_cell

srv = rospy.Service('spawn_srv', Spawn, spawn_turtle)

In [ ]:
jupyros.client('spawn_srv', Spawn)

In [7]:
def get_package_share_directory(package_name):
    """
    Return the share directory of the given package.
    For example, if you install the package 'foo' into
    '/home/user/ros2_ws/install' and you called this function with 'foo' as the
    argument, then it will return '/home/user/ros2_ws/install/share/foo' as
    the package's share directory.
    :param str package_name: name of the package to locate
    :returns: share directory of the package
    :raises: :exc:`PackageNotFoundError` if the package is not found
    """
    return os.path.join(get_package_prefix(package_name), 'share', package_name)

In [16]:
get_package_share_directory("turtlesim")

'/opt/ros/foxy/share/turtlesim'

ModuleNotFoundError: No module named 'ament_index'

In [15]:
from ament_index_python.packages import get_package_share_directory